## Unión de las dos bases de datos

### Instalación requirements

In [2]:
pip install pandas openpyxl xlsxwriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Unión bases

#### Importamos las librerías

In [3]:
import pandas as pd

#### Cargamos los csv

In [6]:
superstore = pd.read_csv("../data/raw/SuperStoreOrders.csv")
world = pd.read_csv("../data/raw/world-cities.csv")

#### Vemos las primeras filas de cada base de datos para saber con que datos vamos a trabajar

In [8]:
print("SuperStore:")
print(superstore.head())

SuperStore:
          order_id order_date ship_date       ship_mode    customer_name  \
0     AG-2011-2040   1/1/2011  6/1/2011  Standard Class  Toby Braunhardt   
1    IN-2011-47883   1/1/2011  8/1/2011  Standard Class      Joseph Holt   
2     HU-2011-1220   1/1/2011  5/1/2011    Second Class    Annie Thurman   
3  IT-2011-3647632   1/1/2011  5/1/2011    Second Class     Eugene Moren   
4    IN-2011-47883   1/1/2011  8/1/2011  Standard Class      Joseph Holt   

       segment            state    country  market   region  ...  \
0     Consumer      Constantine    Algeria  Africa   Africa  ...   
1     Consumer  New South Wales  Australia    APAC  Oceania  ...   
2     Consumer         Budapest    Hungary    EMEA     EMEA  ...   
3  Home Office        Stockholm     Sweden      EU    North  ...   
4     Consumer  New South Wales  Australia    APAC  Oceania  ...   

          category sub_category                 product_name sales quantity  \
0  Office Supplies      Storage          Te

In [9]:
print("\nWorld Cities:")
print(world.head())


World Cities:
        city city_ascii      lat       lng    country iso2 iso3   admin_name  \
0      Tokyo      Tokyo  35.6870  139.7495      Japan   JP  JPN        Tōkyō   
1    Jakarta    Jakarta  -6.1750  106.8275  Indonesia   ID  IDN      Jakarta   
2      Delhi      Delhi  28.6100   77.2300      India   IN  IND        Delhi   
3  Guangzhou  Guangzhou  23.1300  113.2600      China   CN  CHN    Guangdong   
4     Mumbai     Mumbai  19.0761   72.8775      India   IN  IND  Mahārāshtra   

   capital  population          id  
0  primary  37785000.0  1392685764  
1  primary  33756000.0  1360771077  
2    admin  32226000.0  1356872604  
3    admin  26940000.0  1156237133  
4    admin  24973000.0  1356226629  


#### Limpiamos los nombres de país y estado para que coincidan al unirlos

In [10]:
def limpiar_texto(columna):
    return (columna
            .astype(str)
            .str.strip()
            .str.lower()
            .str.normalize("NFKD")  # también quitamos tildes
            .str.encode("ascii", errors="ignore")
            .str.decode("utf-8")
           )

superstore["country"] = limpiar_texto(superstore["country"])
superstore["state"] = limpiar_texto(superstore["state"])
world["country"] = limpiar_texto(world["country"])
world["admin_name"] = limpiar_texto(world["admin_name"])

#### Preparamos una tabla de referencia con datos de world-cities. Agrupamos por país y estado, y calculamos:
- la latitud media
- la longitud media
- la población total

In [11]:
geo_ref = (world.groupby(["country", "admin_name"])
           .agg(latitud=("lat", "mean"),
                longitud=("lng", "mean"),
                poblacion=("population", "sum"))
           .reset_index())

#### Unimos las tablas. Para ello usamos letf join manteniendo todas las filas de SuperStore y añadimos la información de world-cities si coincide el país y estado.

In [12]:
unido = superstore.merge(geo_ref,
                         left_on=["country", "state"],
                         right_on=["country", "admin_name"],
                         how="left")

#### Si algún estado no coincide, usa la media del país como respaldo

In [13]:
faltan = unido["latitud"].isna()
if faltan.any():
    pais_ref = (world.groupby("country")
                .agg(latitud_pais=("lat", "mean"),
                     longitud_pais=("lng", "mean"),
                     poblacion_pais=("population", "sum"))
                .reset_index())
    unido = unido.merge(pais_ref, on="country", how="left")
    # Rellenamos las que faltaban con el promedio del país
    unido["latitud"] = unido["latitud"].fillna(unido["latitud_pais"])
    unido["longitud"] = unido["longitud"].fillna(unido["longitud_pais"])
    unido["poblacion"] = unido["poblacion"].fillna(unido["poblacion_pais"])

#### Guardamos el resultado en un nuevo CSV

In [14]:
unido.to_csv("../data/raw/basededatos_unida.csv", index=False)
print("Unión completa. Archivo guardado como 'basededatos_unida.csv'")

Unión completa. Archivo guardado como 'basededatos_unida.csv'
